# Data Integration 
(Using scANVI, celltypist and GPT-annotations)

In [ ]:
## Define paths
### Human Datasets
path_to_hum = 'path/to/...' # Must be a subset of >300 cells per subtype

### NMR Dataset
path_to_nmr = 'path/to/...' # Using the merge of all regions from cellbender-filtered cells

### Orthology map
path_to_orthologs = '/orthology/nmr_human_orthology.csv' # Merge of biomart nmr->human and human->nmr outputs (1:1 orthologs)

### Annotations
path_to_harmony_crossann = ''
path_to_celltypist_ann = ''
path_to_gpt_ann = ''


In [8]:
import pandas as pd
import scanpy as sc
import scvi
import anndata
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


/usr/local/python/3.12.1/lib/python3.12/site-packages/scvi/__init__.py:31: DeprecationWarning: scvi is deprecated, please uninstall scvi via `pip uninstall scvi` and install the new scvi-tools package at github.com/YosefLab/scvi-tools
  warnings.warn(deprecation_msg, DeprecationWarning)


## Load orthology map

In [ ]:
othologies = pd.read_csv(path_to_orthologs)

## Load Human Data
Using ≥300 cells per subtype (use ~100 per subtype for testing)

In [ ]:
human_adata = sc.read(path_to_hum)
human_adata.obs['species'] = 'human'

## Subset Testing Human Data
Using ~100 cells per subtype

In [ ]:
human_adata.obs

## Load NMR Data


In [ ]:
nmr_adata = sc.read(path_to_nmr)
nmr_adata.obs['species'] = 'nmr'

## Map NMR gene names to human
(mantain NMR gene names from non-orthologs)

## Subset human genes to only 1:1 othologs to NMR

## Merge datasets

# Preprocessing

In [ ]:
# Normalization



0) Assumptions about your adata (what the code expects)

adata is an AnnData with:

.X normalized (you mentioned preprocessing done).

adata.obs['species'] contains 'human' for human cells and 'nmr' (or similar) for naked mole rat cells.

adata.obs['human_label'] exists for human cells (categorical labels at your chosen supertype level); for NMR this can be NaN, 'Unknown' or missing.

The gene set is the intersection of 1:1 orthologs (you already have that).

If names differ, adapt variable names in the code accordingly.

In [ ]:
# make a copy so we can revert if needed
adata_sc = adata.copy()

# 1) Create a labels column that scANVI can use. 
#    We want human cells labeled and NMR unlabeled (set to 'Unknown').
adata_sc.obs['scvi_label'] = adata_sc.obs.get('human_label', None).astype('category')

# Set unlabeled category name
unlabeled_cat = 'Unknown'
adata_sc.obs['scvi_label'] = adata_sc.obs['scvi_label'].cat.add_categories([unlabeled_cat])
adata_sc.obs.loc[adata_sc.obs['species'] != 'human', 'scvi_label'] = unlabeled_cat

# If you have a batch key (donor, library, region) include it; else leave out.
# Example: use 'batch' if exists
batch_key = 'batch' if 'batch' in adata_sc.obs.columns else None

# Register anndata with scvi-tools
if batch_key:
    scvi.data.setup_anndata(adata_sc, labels_key='scvi_label', batch_key=batch_key)
else:
    scvi.data.setup_anndata(adata_sc, labels_key='scvi_label')


## Train scANVI
Use human annotated cells and include NMR as unlabeled

In [ ]:
from scvi.model import SCANVI

# Create model: tell it the unlabeled category name (the string we used above)
model = SCANVI(
    adata_sc,
    unlabeled_category=unlabeled_cat,
    labels_key='human_label',
    n_latent=20,
    n_layers=2,
    n_hidden=128,
    dropout_rate=0.1
)

# Train on CPU: set max_epochs (start modest)
max_epochs = 120
model.train(max_epochs=max_epochs, use_gpu=False)


## Get scANVI predictions

In [ ]:
# 1) hard predicted labels
pred_labels = model.predict(adata_sc)  # returns array of category strings
adata_sc.obs['scanvi_pred'] = pred_labels

# 2) probabilities: try the convenience call first
try:
    probs = model.predict(adata_sc, soft=True)  # returns np.array shape (n_cells, n_classes)
    # If the returned structure is a DataFrame, convert to numpy
    if isinstance(probs, pd.DataFrame):
        probs = probs.values
except TypeError:
    # fallback: some versions provide predict_proba()
    try:
        probs = model.predict_proba(adata_sc)
    except Exception:
        # ultimate fallback: compute probabilites from the posterior logits
        print("predict(..., soft=True) and predict_proba() failed; attempting posterior sampling fallback.")
        posterior = model.get_posterior(adata_sc)
        probs = posterior.label_probabilities().numpy()  # may work in newer versions


In [ ]:
# get label names in the same order as columns of probs
label_names = model.summary_labels_  # scvi usually stores label names here; fallback below
if label_names is None:
    # fallback to categories from adata
    label_names = adata_sc.obs['scvi_label'].cat.categories.tolist()

# Convert to DataFrame and attach
probs_df = pd.DataFrame(probs, columns=label_names, index=adata_sc.obs_names)
# top probability and argmax label
adata_sc.obs['scanvi_top_prob'] = probs_df.max(axis=1).values
adata_sc.obs['scanvi_top_label'] = probs_df.idxmax(axis=1).values

# Optional: store full probs in adata.obsm
adata_sc.obsm['scanvi_proba'] = probs


## Integrate different annotations
For each cell compute:

scANVI predicted label + probability (softmax).

Marker enrichment score for the predicted label: run DE (cluster vs others or cell vs others) and compute AUC for canonical markers. For marker lists, use human canonical markers mapped via orthology.

celltypist prediction + probability (if celltypist supports the species/model).

GPT-marker annotation score (you already have these — convert to binary match vs canonical markers).

Cluster consensus: what fraction of cells in the cluster share the predicted label.

Combine into a consensus score per cell, e.g.:

consensus_score = 0.5 * scANVI_prob + 0.3 * marker_enrichment_score + 0.2 * celltypist_prob

(Weights are configurable; I prefer classifier probability highest.) Normalize to [0,1].

## Assign final labels
Consider label assignment only when confidence threshold is passed

If consensus_score >= 0.8 → assign label.

If 0.5 <= consensus_score < 0.8 → assign label but mark "provisional / review".

If consensus_score < 0.5 → label as "unknown" or “species-specific candidate.”
Report how many cells fall in each bin and list “unknown” clusters for manual curation.

For each cluster, compute:

Label purity (fraction of cells with the same final label).

Mean scANVI probability and mean marker enrichment.

Top DE genes (pseudobulk) and compare to known marker panels.

For clusters with low purity or conflicting evidence, either:

Split cluster and re-run scANVI/probabilities, or

Keep as “ambiguous / novel” and provide marker list + suggested experiments (ISH, immuno).

In [ ]:
sc.tl.rank_genes_groups(method='wilcoxon')

UMAPs colored by each annotation method and by consensus_label and consensus_score (continuous heat).

Confusion matrices between methods (human cross-annotation vs celltypist vs scANVI).

Alluvial plot showing how cells move between annotations.

Per-label ROC / PR curves for marker enrichment vs assigned label.

In [ ]:
sc.pl.umap(adata, color=['consensus_label','scANVI_prob'])

Make a table with one row per final label containing:

number cells, purity, consensus_score_mean, canonical markers enriched, top DE genes, suggested validation (marker probes), and confidence (high/medium/low).